In [1]:
import pynwb
from allensdk.brain_observatory.ecephys.dynamic_gating_ecephys_session import DynamicGatingEcephysSession
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
%matplotlib inline

import xarray as xr
import sys
import pickle

# Add the path you want to include
sys.path.append('code')

from shield_utils import find_animals, get_lfp_dict, downsample, align_lfp, load_animals_oi

import json

data_path = '/oscar/data/sjones/kduecker/shield_data/'
meta_path = 'externals/SHIELD_Dynamic_Gating_Analysis'

toi = [0, 2]


In [27]:
mice_sess = load_animals_oi()


# load the sessions that have the ROIs

df = pd.DataFrame()

for subj in list(mice_sess.keys()):

    ses_files = os.listdir(os.path.join(data_path,f'sub-{subj}'))           # sessions per mouse

    for session in mice_sess[subj]:

        df_subj = pd.DataFrame()

        print(f'loading session: {session}', flush=True)
        ses_file = list(filter(lambda s: session in s, ses_files))

        nwb_file_asset = pynwb.NWBHDF5IO(f'{data_path}/sub-{subj}/{ses_file[0]}', mode='r', load_namespaces=True)
        nwb_file = nwb_file_asset.read()
        dynamic_gating_session = DynamicGatingEcephysSession.from_nwb(nwb_file)

        eye_tracking = dynamic_gating_session.eye_tracking
        running_speed = dynamic_gating_session.running_speed
        licks = dynamic_gating_session.licks

        # get stimulus presentations
        stim_presentations = dynamic_gating_session.stimulus_presentations
        flashes = stim_presentations[stim_presentations['stimulus_name'].str.contains('flash')]
        presentation_times = flashes.start_time.values
        flash_end_times = presentation_times + flashes.duration
        presentation_ids = flashes.index.values

        mean_speed = np.zeros_like(presentation_times)
        num_licks = np.zeros_like(presentation_times)
        num_rewards = np.zeros_like(presentation_times)
        num_blinks = np.zeros_like(presentation_times)

        for i,trial_start in enumerate(presentation_times):

            # mean running speed
            flash_running = running_speed.query('timestamps >= {} and timestamps <= {} '.format(trial_start-toi[0], trial_start+toi[1]))
            mean_speed[i] = np.mean(flash_running.speed.values)
            # number of licks
            flash_licks = licks.query('timestamps >= {} and timestamps <= {} '.format(trial_start-toi[0], trial_start+toi[1]))
            num_licks[i] = len(flash_licks)
            # check if this makes sense: rewards?
            reward_time = dynamic_gating_session.rewards.query('timestamps >= {} and timestamps <= {} '.
                                            format(trial_start-toi[0], trial_start+toi[1]))['timestamps']

            num_rewards[i] = len(reward_time)

            # number of blinks
            eye_blinks = eye_tracking[eye_tracking['likely_blink']].query('timestamps >= {} and timestamps <= {} '.format(trial_start-toi[0], trial_start+toi[1]))
            num_blinks[i] = len(eye_blinks)

            df_subj['subject_id'] = 'sub-'+str(subj)
            df_subj['onset_time'] = presentation_times

            df_subj['mean_speed'] = mean_speed
            df_subj['num_licks'] = num_licks
            df_subj['num_rewards'] = num_rewards
            df_subj['num_blinks'] = num_blinks

            for meta in dynamic_gating_session.metadata:
                if type(dynamic_gating_session.metadata[meta]) == list:
                    df_subj[meta] = dynamic_gating_session.metadata[meta][0]
                else:
                    df_subj[meta] = dynamic_gating_session.metadata[meta]

        df = pd.concat([df,df_subj])


df.to_csv(os.path.join(data_path,'results_behavior',f'behav_all.csv'))



loading session: 0220607


loading session: 0220609
loading session: 0220608
loading session: 0220928


/users/kduecker/.conda/envs/allensdk/lib/python3.9/site-packages/allensdk/brain_observatory/behavior/data_objects/metadata/subject_metadata/full_genotype.py:57: UserWarning: Unable to parse cre_line from full_genotype
  warnings.warn('Unable to parse cre_line from full_genotype')
/users/kduecker/.conda/envs/allensdk/lib/python3.9/site-packages/allensdk/brain_observatory/behavior/data_objects/metadata/subject_metadata/full_genotype.py:57: UserWarning: Unable to parse cre_line from full_genotype
  warnings.warn('Unable to parse cre_line from full_genotype')
/users/kduecker/.conda/envs/allensdk/lib/python3.9/site-packages/allensdk/brain_observatory/behavior/data_objects/metadata/subject_metadata/full_genotype.py:57: UserWarning: Unable to parse cre_line from full_genotype
  warnings.warn('Unable to parse cre_line from full_genotype')
/users/kduecker/.conda/envs/allensdk/lib/python3.9/site-packages/allensdk/brain_observatory/behavior/data_objects/metadata/subject_metadata/full_genotype.py:

loading session: 0220926


/users/kduecker/.conda/envs/allensdk/lib/python3.9/site-packages/allensdk/brain_observatory/behavior/data_objects/metadata/subject_metadata/full_genotype.py:57: UserWarning: Unable to parse cre_line from full_genotype
  warnings.warn('Unable to parse cre_line from full_genotype')
/users/kduecker/.conda/envs/allensdk/lib/python3.9/site-packages/allensdk/brain_observatory/behavior/data_objects/metadata/subject_metadata/full_genotype.py:57: UserWarning: Unable to parse cre_line from full_genotype
  warnings.warn('Unable to parse cre_line from full_genotype')
/users/kduecker/.conda/envs/allensdk/lib/python3.9/site-packages/allensdk/brain_observatory/behavior/data_objects/metadata/subject_metadata/full_genotype.py:57: UserWarning: Unable to parse cre_line from full_genotype
  warnings.warn('Unable to parse cre_line from full_genotype')
/users/kduecker/.conda/envs/allensdk/lib/python3.9/site-packages/allensdk/brain_observatory/behavior/data_objects/metadata/subject_metadata/full_genotype.py:

loading session: 0221110


/users/kduecker/.conda/envs/allensdk/lib/python3.9/site-packages/allensdk/brain_observatory/behavior/data_objects/metadata/subject_metadata/full_genotype.py:57: UserWarning: Unable to parse cre_line from full_genotype
  warnings.warn('Unable to parse cre_line from full_genotype')
/users/kduecker/.conda/envs/allensdk/lib/python3.9/site-packages/allensdk/brain_observatory/behavior/data_objects/metadata/subject_metadata/full_genotype.py:57: UserWarning: Unable to parse cre_line from full_genotype
  warnings.warn('Unable to parse cre_line from full_genotype')
/users/kduecker/.conda/envs/allensdk/lib/python3.9/site-packages/allensdk/brain_observatory/behavior/data_objects/metadata/subject_metadata/full_genotype.py:57: UserWarning: Unable to parse cre_line from full_genotype
  warnings.warn('Unable to parse cre_line from full_genotype')


loading session: 0221108
loading session: 0221201
loading session: 0220812
loading session: 0220810


In [29]:
df = df.rename(columns={df.columns[0]: 'trial'})

In [25]:
df.to_csv(os.path.join(f'behav_all.csv'))


In [30]:
df

,trial,onset_time,mean_speed,num_licks,num_rewards,num_blinks,equipment_name,sex,age_in_days,stimulus_frame_rate,...,date_of_acquisition,reporter_line,cre_line,behavior_session_uuid,driver_line,mouse_id,project_code,full_genotype,behavior_session_id,ecephys_session_id
0,sub-607660,5189.394926,8.953436,0.0,0.0,0.0,NP.1,F,225,60.0,...,2022-06-07 21:25:34.215000+00:00,Ai32(RCL-ChR2(H134R)_EYFP),Sst-IRES-Cre,3bbedd1d-cf74-492b-b20a-14d6bc5cd3dd,Sst-IRES-Cre,607660,Not Available,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,1182661832,1182628226
1,sub-607660,5191.396746,9.336086,0.0,0.0,0.0,NP.1,F,225,60.0,...,2022-06-07 21:25:34.215000+00:00,Ai32(RCL-ChR2(H134R)_EYFP),Sst-IRES-Cre,3bbedd1d-cf74-492b-b20a-14d6bc5cd3dd,Sst-IRES-Cre,607660,Not Available,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,1182661832,1182628226
2,sub-607660,5193.398416,12.062332,0.0,0.0,0.0,NP.1,F,225,60.0,...,2022-06-07 21:25:34.215000+00:00,Ai32(RCL-ChR2(H134R)_EYFP),Sst-IRES-Cre,3bbedd1d-cf74-492b-b20a-14d6bc5cd3dd,Sst-IRES-Cre,607660,Not Available,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,1182661832,1182628226
3,sub-607660,5195.399976,10.902618,0.0,0.0,0.0,NP.1,F,225,60.0,...,2022-06-07 21:25:34.215000+00:00,Ai32(RCL-ChR2(H134R)_EYFP),Sst-IRES-Cre,3bbedd1d-cf74-492b-b20a-14d6bc5cd3dd,Sst-IRES-Cre,607660,Not Available,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,1182661832,1182628226
4,sub-607660,5197.401636,11.806524,0.0,0.0,0.0,NP.1,F,225,60.0,...,2022-06-07 21:25:34.215000+00:00,Ai32(RCL-ChR2(H134R)_EYFP),Sst-IRES-Cre,3bbedd1d-cf74-492b-b20a-14d6bc5cd3dd,Sst-IRES-Cre,607660,Not Available,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,1182661832,1182628226
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,sub-615048,5552.867282,36.354786,0.0,0.0,0.0,NP.1,M,233,60.0,...,2022-08-10 21:14:53.846000+00:00,Ai32(RCL-ChR2(H134R)_EYFP),Vip-IRES-Cre,4d93b60e-5f05-40bc-83ab-00a3a859ad8d,Vip-IRES-Cre,615048,Not Available,Vip-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,1197751799,1197725774
146,sub-615048,5554.868872,7.686500,0.0,0.0,0.0,NP.1,M,233,60.0,...,2022-08-10 21:14:53.846000+00:00,Ai32(RCL-ChR2(H134R)_EYFP),Vip-IRES-Cre,4d93b60e-5f05-40bc-83ab-00a3a859ad8d,Vip-IRES-Cre,615048,Not Available,Vip-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,1197751799,1197725774
147,sub-615048,5556.870632,26.215525,0.0,0.0,0.0,NP.1,M,233,60.0,...,2022-08-10 21:14:53.846000+00:00,Ai32(RCL-ChR2(H134R)_EYFP),Vip-IRES-Cre,4d93b60e-5f05-40bc-83ab-00a3a859ad8d,Vip-IRES-Cre,615048,Not Available,Vip-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,1197751799,1197725774
148,sub-615048,5558.872182,8.715854,0.0,0.0,0.0,NP.1,M,233,60.0,...,2022-08-10 21:14:53.846000+00:00,Ai32(RCL-ChR2(H134R)_EYFP),Vip-IRES-Cre,4d93b60e-5f05-40bc-83ab-00a3a859ad8d,Vip-IRES-Cre,615048,Not Available,Vip-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,1197751799,1197725774
